In [1]:
# Importing Necessary Libraries
from os import listdir

# For preprocessing
from nltk.corpus import stopwords
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
# import unidecode


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
# !unzip  /content/drive/MyDrive/Humor,Hist,Media,Food.zip -d /content/drive/MyDrive/IR_A1
# !pip install contractions
!pip install unidecode

     |████████████████████████████████| 235 kB 5.0 MB/s 


In [3]:
# Mount the drive for loading data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Load the files and stop words
FILE_PATH = "/content/drive/MyDrive/IR_A1/Humor,Hist,Media,Food/"
stopWords = stopwords.words('english')

In [5]:
def remove_punctuation(data):
    '''
    input-> data (text corpus/ input sentence)
    output-> data without punctuations
    '''

    # List of punctuations to remove
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    
    # Check and replace each punctuations
    for i in range(len(symbols)):

        data = data.replace(symbols[i], ' ')
        data = data.replace("  ", " ")
    
    # Remove commas separately
    data = data.replace(',', ' ')
    
    return data

def preprocessing(data):
    '''
    input-> data (text corpus/ input sentence)
    output-> tokens (preprocesses data)
    '''
    
    # Convert the text to lower case 
    data = data.lower() 

    data = remove_punctuation(data)

    # Word Tokenizing
    tokens = nltk.word_tokenize(data)
    
    pTokens = []

    # Remove Stopwords, punctuation marks and blank tokens
    for word in tokens:
        # For each word check the following: extra spaces, punctuations, stop words and numbers
        if (word.isalpha() and word not in stopWords):

            # Append the correct words
            pTokens.append(word.strip().translate(str.maketrans('', '', string.punctuation)))
        
    # Return the final tokens
    return pTokens

In [6]:
def get_data(file_name):
    '''
    input-> Name of file (assumption: exists in the corpus)
    output-> The written data in the file (unprocessed)
    '''

    # Open a file
    with open(FILE_PATH + file_name, 'r') as f:

        # Return the complete data
        return f.read()

In [7]:
def positionalIndexing():
    '''
    input-> None
    output-> A dictionary of suitable size and structure containing the positional indexes of all the data
    '''

    # Get all the file names in the dataset
    FILE_LIST = listdir(FILE_PATH)

    # Initialize an empty dictionary
    positionalIndex = {}
    
    # Run a loop
    for docID in range(len(FILE_LIST)):

        # Get the name of the document
        document = FILE_LIST[docID]

        # Try to read the data, and handeling exceptions
        try:

            # preprocess the data from the document 
            tokens = list(preprocessing(get_data(document)))

        # exception
        except UnicodeDecodeError:

            # preprocess the data from the document 
            data = open(FILE_PATH + document, 'rb').read().decode('ISO-8859-1')

            # List all the tokens
            tokens = list(preprocessing(data))
        
        # For each token in the processed file
        position = 0
        for token in tokens:

            # Check if the token is already present in the dictionary
            if token in positionalIndex.keys():

                # increment the frequency
                positionalIndex[token][0] += 1
            
                # Check if the document is present in the dictionary
                if docID in positionalIndex[token][1].keys():

                    # If present, increment the in - doc frequency and add the new position to list of positions
                    positionalIndex[token][1][docID][0] += 1
                    positionalIndex[token][1][docID].append(position)

                else:

                    # If doc ID not present, add the document as a key and the postion in a new list
                    positionalIndex[token][1][docID] = [1, position]
                position += 1
           
            else:

                # If the token is not present, add the token, document ID and position
                positionalIndex[token] = [1, {docID:[1, position]}]

                # Increment the position for the next token
                position += 1


    # Return the positional index dictionary
    return positionalIndex


def getDocName(docID):
    '''
    input-> Doc ID (index of document in FILE_LIST)
    output-> The name of the document as in the folder.
    '''

    # Get the file list and return the name using the index
    FILE_LIST = listdir(FILE_PATH)
    return FILE_LIST[docID]



In [8]:
# Calling the function to make the positional indexes
pDict = positionalIndexing()

In [11]:

def preprocessInput(inputWord):
    '''
    input-> Input query
    output-> Preprocessed query
    '''

    # Use the same preprocessing as the data
    return preprocessing(inputWord)



def getWordfrequency(preprocessedWord):
    '''
    input-> Preprocessed query
    output-> True if the word is present in dictionary, False otherwise
    '''
    
    # Check if the word is present in the pDict
    if preprocessedWord in pDict.keys():

        # Get the frequency
        freq = pDict[preprocessedWord][0]

        # Return the flag and frequency
        return True, freq

    # Return False if the word is not present
    return False, 0



def getListNames(docIDs): 
    '''
    input-> List/ Set of docIds
    output-> List of name of the documents
    '''
   
    docNames = []

    # Check the type of input depending on the length of input query
    if type(docIDs) == set:

        # Get the name of each document
        for docId in docIDs:
            docNames.append(getDocName(docId))
        
        # Return the names
        return docNames
    
    # If the input is dictionary
    for docId in docIDs.keys():
        docNames.append(getDocName(docId))
    
    return docNames

    
        

def getIndexes(input):
    '''
    input-> Input query
    output-> print the number of matched documents and their names. Return the completion statement of search.
    '''

    # Process the input query
    tokens = preprocessInput(input)

    # Check length of tokens
    if (len(tokens) == 0):
        return "No words left (after preprocessing)"
    
    # Check if all the words are present in the dictionary
    for word in tokens:
        
        # Check if any token is not available in the dictionary, return
        flag, wordFrequency = getWordfrequency(word)

        if not flag:
            return "Word(s) unavailable"

                
    
    # get the frequency of initial word for output
    flag, frequency = getWordfrequency(tokens[0])
    
    # Otherwise, check if the length of input is 1. Return all the documents with the token
    if len(tokens) == 1:

        # All the documents
        docs = pDict[tokens[0]][1]

        print("Search Complete")
        print("The number of documents retrieved :", frequency)

        # Print the names of the documents
        print("The list of documents names ")
        print(getListNames(docs))

        
    # If the input query is longer than 1.
    else:

        # Get the documents that contain the first word
        startWord = tokens[0]
        startDocs = pDict[startWord][1]
        docs = []
        
        # For all the documents in the position list of first word
        for doc in startDocs.keys():
            docFreq = startDocs[doc][0]

            # Get the position of first token in the document
            docList = startDocs[doc][1:]

            wordCount = 0

            # For each position present
            for position in docList:
                posn = position

                # Check if all the next tokens are present at the subsequent positions
                for word in tokens[1:]:

                    # Increment the position
                    posn += 1

                    # Check if the next token is present in the document, if present check if the token is present at correct position
                    if doc in pDict[word][1].keys() and posn in pDict[word][1][doc]:

                        # If present, add the document
                        docs.append(doc)

                    else:

                        # Check the next positions
                        break

            
        # Return the frequency and names
        print("Search Complete")

        # Take the documents only once
        print("The number of documents retrieved :", len(set(docs)))
        print("The list of documents names ")
        print(getListNames(set(docs)))


    # Search complete
    return "Program Terminated"

In [15]:
print("\n--Phrase Query Search--\n")

# Enter the input
inputSearch = input('Enter the Phrase\n')
print()
# inputSearch = 'first aid'

# Initiate the search for the input query
getIndexes(inputSearch)



--Phrase Query Search--

Enter the Phrase




'No words left (after preprocessing)'